In [113]:
# importing libraries 
import numpy as np
from attention_utils import get_data # scirpt to genetate data
import pandas as pd
#from __future__ import print_function
# import keras
import keras
#keras.__version__
print(keras.__version__)
from keras.layers import Dense
from keras.models import Sequential
from keras.models import Model
# all modules of keras for lSTM-RNNs
from keras.models import Sequential
from keras.layers import LSTM
#from keras.layers import Merge # problem with Merge because I now use Keras 2.2.4. Merge is replaced by add
from keras.layers import add
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop,Adamax
from keras import backend as K

# turn off the warnings, be careful when use this
import warnings
warnings.filterwarnings("ignore")

2.2.4


In [114]:
# Reading files
# features
X_test=pd.read_csv('Test_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None) # reading ok #X_test.values
X_train=pd.read_csv('Train_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None)
X_valid=pd.read_csv('Valid_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None)
# labels
Y_test=pd.read_csv('Test_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
Y_train=pd.read_csv('Train_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
Y_valid=pd.read_csv('Valid_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
# small sets
X_test_s=X_test[0:101]
X_train_s=X_train[0:201]
X_valid_s=X_valid[0:101]
Y_test_s=Y_test[0:101]
Y_train_s=Y_train[0:201]
Y_valid_s=Y_valid[0:101]

In [115]:
number_of_features = X_train.shape[1] # columns are features , rows are observations
print(number_of_features)
number_of_observations = X_train.shape[0] # columns are features , rows are observations
print(number_of_observations)

1100
19428


In [116]:
# function to reshape the panda.DataFrame format data to Keras style: (batch_size, time_step, nb_features)
def reshape_data(data, n_prev): #n_prev = time_step 
    docX = []
    for i in range(len(data)):
        if i < (len(data)-n_prev):
            docX.append(data.iloc[i:i+n_prev].as_matrix())
        else: # the frames in the last window use the same context
            docX.append(data.iloc[(len(data)-n_prev):len(data)].as_matrix())
    alsX = np.array(docX)
    return alsX

In [117]:
# custom evaluation metrics
def pearson_cc(y_true, y_pred):
    fsp = y_pred - K.mean(y_pred,axis=0)   
    fst = y_true - K.mean(y_true,axis=0) 
    devP = K.std(y_pred,axis=0)  
    devT = K.std(y_true,axis=0)

    return K.sum(K.mean(fsp*fst,axis=0)/(devP*devT))

In [118]:
# Set up the model: model
def build_LSTM_model(time_step, nb_features, H1, dropout_W1, dropout_U1,H2, dropout_W2, dropout_U2, H3, dropout_W3, dropout_U3):
    # define variables
# nb_features = 1000 # total number of features, also the number of neurons in the input layer of LSTM
# time_step = 3  # the length of history (number of previous data instances) to include
# H1 = 128 # number of neurons in the bottom hidden layer
# H2 = 64 # number of neurons in the middle hidden layer
# H3 = 32 # number of neurons in the top hidden layer
# dropout_W1 = 0.5 # drop out weight (for preventing over-fitting) in H1
# dropout_U1 = 0.5 # drop out weight (for preventing over-fitting) in H1
# dropout_W2 = 0 # drop out weight (for preventing over-fitting) in H2
# dropout_U2 = 0 # drop out weight (for preventing over-fitting) in H2
# dropout_W3 = 0 # drop out weight (for preventing over-fitting) in H3
# dropout_U3 = 0 # drop out weight (for preventing over-fitting) in H3
    # define model structure
    model = Sequential()
    model.add(LSTM(H1, input_shape=(time_step, nb_features),dropout_W=dropout_W1, dropout_U=dropout_U1, return_sequences=True)) 
    # bottom hidden layer
    model.add(LSTM(H2, dropout_W=dropout_W2, dropout_U=dropout_U2, return_sequences=True)) # middle hidden layer
    model.add(LSTM(H3, dropout_W=dropout_W3, dropout_U=dropout_U3, return_sequences=False)) # top hidden layer
    model.add(Dense(1, activation='sigmoid')) # output layer, regression task, value range [-1,1]
    return model


In [119]:
# optimization algorithms
def opt_LSTM_model(lr):
    #opt_func = RMSprop(lr=0.0001) # training function
    # lr=0.0005 # learning rate for the optimization algorithm 
    opt_func = Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    return opt_func

In [120]:
# define variables
nb_features = number_of_features # total number of features, also the number of neurons in the input layer of LSTM
time_step = 3  # the length of history (number of previous data instances) to include
batch_size = 32 # the model performs a weight update for every batch, smaller batch means faster learning but les stable weights
nb_epoch = 1000 # number of total epochs to train the model
H1 = 128 # number of neurons in the bottom hidden layer
H2 = 64 # number of neurons in the middle hidden layer
H3 = 32 # number of neurons in the top hidden layer
dropout_W1 = 0.5 # drop out weight (for preventing over-fitting) in H1
dropout_U1 = 0.5 # drop out weight (for preventing over-fitting) in H1
dropout_W2 = 0 # drop out weight (for preventing over-fitting) in H2
dropout_U2 = 0 # drop out weight (for preventing over-fitting) in H2
dropout_W3 = 0 # drop out weight (for preventing over-fitting) in H3
dropout_U3 = 0 # drop out weight (for preventing over-fitting) in H3
lr=0.0005 # learning rate for the optimization algorithm 


# function that merge testing, validation, optimzation settings, and compilation
def train_valid_model(nb_features,time_step,batch_size,nb_epoch,H1, dropout_W1, dropout_U1,H2, dropout_W2, dropout_U2, H3, dropout_W3, dropout_U3,lr):

    LSTM_model = build_LSTM_model(time_step, nb_features, H1, dropout_W1, dropout_U1,H2, dropout_W2, dropout_U2, H3, dropout_W3, dropout_U3)
    opt_func = opt_LSTM_model(lr)
    # Compiling
    LSTM_model.compile(loss=pearson_cc, optimizer=opt_func, metrics=[pearson_cc,'mse']) 
    # define the optimizer for training, use cc and mse as the evaluation metric
    # carry out training

    # Reshaping the panda.DataFrame format data to Keras style: (nb_observations, time_step, nb_features)
    # shape (nb_observations, nb_features)
    #X_Test = reshape_data(X_test)
    #X_Train = reshape_data(X_train)
    #X_Valid = reshape_data(X_valid)
    # small set
    X_Test = reshape_data(X_test_s,time_step)
    X_Train = reshape_data(X_train_s,time_step)
    X_Valid = reshape_data(X_valid_s,time_step)
    # after reshaping: shape (nb_observations,time_step,nb_features)

    # if the validation loss isn't decreasing for a number of epochs, stop training to prevent over-fitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    LSTM_model.fit(X_Train, Y_train_s, batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_Valid, Y_valid_s), callbacks=[early_stopping], verbose=0)
    return LSTM_model

# training the model using validation set
LSTM_model=train_valid_model(nb_features,time_step,batch_size,nb_epoch,H1, dropout_W1, dropout_U1,H2, dropout_W2, dropout_U2, H3, dropout_W3, dropout_U3,lr)

In [121]:
def test_model(LSTM_model, X_Test, Y_test, batch_size):
    # evaluation
    test_score, test_cc, test_mse = LSTM_model.evaluate(X_Test, Y_test, batch_size=batch_size, verbose=0)
    print('Test cc:', test_cc)
    print('Test mse:', test_mse)
    print('\n')
    # output predictions
    test_prediction = LSTM_model.predict(X_Test)
    test_prediction_df = pd.DataFrame(test_prediction)
    return test_prediction_df


In [122]:
# output predictions
test_pred_df=test_model(LSTM_model,X_Test, Y_test_s,batch_size)
test_pred_df.to_csv('prediction.csv', mode='a', index=False, header=False)
print(test_pred_df)

Test cc: 0.04961800634270847
Test mse: 0.00949685377216354


            0
0    0.467780
1    0.472603
2    0.477776
3    0.481069
4    0.478078
5    0.476572
6    0.477086
7    0.483431
8    0.487802
9    0.483441
10   0.479389
11   0.480100
12   0.483656
13   0.487532
14   0.489766
15   0.489046
16   0.488323
17   0.491141
18   0.494248
19   0.488238
20   0.483318
21   0.480842
22   0.488443
23   0.499180
24   0.506261
25   0.503984
26   0.498233
27   0.492405
28   0.485391
29   0.479225
..        ...
71   0.501757
72   0.503246
73   0.504472
74   0.504564
75   0.509212
76   0.513236
77   0.512897
78   0.509642
79   0.500684
80   0.489340
81   0.492384
82   0.507265
83   0.517534
84   0.517531
85   0.507962
86   0.497797
87   0.492670
88   0.490747
89   0.489687
90   0.487853
91   0.485782
92   0.482967
93   0.482443
94   0.481944
95   0.480962
96   0.482148
97   0.489965
98   0.500279
99   0.500279
100  0.500279

[101 rows x 1 columns]
